# Set up Imports and Libraries

In [2]:
# === Import Necessary Libraries ===
# These are core scientific Python tools for data analysis and visualization.

import matplotlib.pyplot as plt
import numpy as np
from open_ephys.analysis import Session
import os
import re
from scipy.signal import butter, filtfilt, lfilter
from collections import defaultdict
import scipy.stats as stats


# 1. Selecting and Exploring a Recording Session

Open Ephys organizes experiments into “sessions,” each with one or more Record Nodes.
Here we select which node to analyze and define which session directory to load.

----------------

### Node Selection Raw vs. Filtered Node in Open Ephys

In this section we define a few arbitrary floating variables for processing purposes to help us select the right recording nodes we want, these variables will be "w", and "u".

#### In this case...

##### **NODE 111 = The RAW Node with Raw-unfiltered data** (Index = 0)

##### **NODE 106 = The FILTERD Node with Online-filtered data.** (Index = 1)

> Setting "u" and "w" equal to zero, wll result in selecting the first node **(0 = Record Node 106/RAW)**

> Setting "u" and "w" equal to one, wll result in selecting the second node **(1 = Record Node 111/FILTERED)**

In [6]:
# ==== Session Selection ====
w = 0  # Record Node index (0 = Record Node 106)
u = 0  # Record Node index (unused, reserved for alternate node, set to same as "w")

### Function: Scanning Experiment Structure

We’ll define a helper function to automatically list all experiments and recordings inside a session directory.

In [9]:
def scan_experiment_structure(session_dir, record_node_name="Record Node 106"):
    """
    Scans a given Open Ephys session directory and returns a dictionary of 
    experiment names and their associated recordings.
    """
    node_path = os.path.join(session_dir, record_node_name)
    
    if not os.path.exists(node_path):
        print(f"Record Node directory not found: {node_path}")
        return {}
    
    experiment_info = defaultdict(list)
    for item in sorted(os.listdir(node_path)):
        exp_path = os.path.join(node_path, item)
        if os.path.isdir(exp_path) and item.startswith("experiment"):
            recordings = [r for r in sorted(os.listdir(exp_path)) 
                          if os.path.isdir(os.path.join(exp_path, r)) and r.startswith("recording")]
            experiment_info[item] = recordings
    return experiment_info


# 2. Loading the Session Data

Now let’s point Python to a specific Open Ephys session directory and create a Session object.

In [8]:
# ==== Load Session ====
directory = 'HRPilot-05_FirstChronicTest1_2025-08-27_17-17-00_001'

session = Session(directory)
print('✅ Pass Initial Test, Moving on to Loading Session...\n')

✅ Pass Initial Test, Moving on to Loading Session...



### Scanning the Session's Structure

> Observe how the floating variable "w" we previously defined earlier is used here to 

In [11]:
record_node_name = "Record Node 106" if w == 0 else "Record Node 111"
structure = scan_experiment_structure(directory, record_node_name)

# Flatten all experiment-recording pairs
flat_recordings = []
for exp_name, rec_list in structure.items():
    for rec in rec_list:
        flat_recordings.append((exp_name, rec))

print(f"Found {len(flat_recordings)} recordings across {len(structure)} experiments.")


Found 1 recordings across 1 experiments.
